In [50]:
from queue import PriorityQueue
import sys

class Graph:
    def __init__(self, edges): #edges presented as tuples
        self.adjList = dict() #adjacency list representation
        self.edges = edges
        for (u, v), weight in edges.items():
            if u in self.adjList:
                self.adjList[u].append((v, weight)) 
            else:
                self.adjList[u] = [(v, weight)]
            if v in self.adjList:
                self.adjList[v].append((u, weight))
            else:
                self.adjList[v] = [(u, weight)]
        self.N = len(self.adjList)
        
    def kruskalMST(self):
        self.mstEdges = []
        pq = PriorityQueue()
        cost = 0
        for (u, v), weight in self.edges.items(): #for each edge
            pq.put((weight, (u, v)))
        uf = UnionFind(self.N)
        while (not pq.empty()) and (len(self.mstEdges) < self.N - 1):
            e = pq.get()
            u = e[1][0]
            v = e[1][1]
            if not uf.findCycle(u, v):
                uf.union(u, v)
                self.mstEdges.append(e)
                cost += e[0]
        return (self.mstEdges, cost)
    
    def primMST(self):
        self.mstEdges = []
        marked = dict()
        cost = 0
        root = None
        for u in self.adjList.keys():
            if root is None:
                root = u
            marked[u] = False
            
        pq = {root : [0, None]}
        while pq:
            # choose adj edge with the least edge cost, remove it from the priority q
            u = min(pq, key = lambda x: pq[x][0])
            weight = pq[u][0]
            v = pq[u][1]
            del pq[u]
            
            if marked[u] == False :
                cost = cost + weight
                marked[u] = True
                if v is not None:
                    self.mstEdges.append((weight, (u, v)))
                for (x, w) in self.adjList[u]:
                    if marked[x] == False:
                        pq[x] = (w, u)
        return (self.mstEdges, cost)
    
    #simplest algorithm to check whether edge is in MST
    def isEdgeInMST(self, edge):
            u = edge[0]
            v = edge[1]
            w = edge[2]
            if (u, v, w) in self.mstEdges:
                return True
            else:
                return False
            
    #linear time 
    #graph G' containing edges w weight < e (source, dest, w). we now check whether they're are still connected; if they are, then e does not belong to any mst
    def isEdgeInMSTLinear(self, edge):
        source = edge[0]
        dest = edge[1]
        maxweight = edge[2]
        visited = dict()      
        visited[source] = True
        q = deque()
        q.append(source)
        while len(q)>0:
            u = q.popleft()
            #for every edge u --> (v, w)
            for v, w in graph.adjList[u]:
                #if u has not been visited yet
                if v not in visited and w < maxweight:
                    visited[v] = True
                    q.append(v)
        if (source in visited) and (dest in visited):
            return False
        else:
            return True
        
    def relax(self, u, v, w, edgeTo, distTo, pq):
            if distTo[v] > distTo[u] + w:
                distTo[v] = distTo[u] + w
                edgeTo[v] = (u,v,w) #latest edge in shortest path to v
                pq[v] = distTo[v]
    
    def dijkstra(self, source):
        edgeTo = dict()
        distTo = dict()

        for k in self.adjList.keys():
            edgeTo[k] = None
            distTo[k] = sys.maxsize
        distTo[source] = 0

        pq = {source : 0}
        while pq:
            u = min(pq, key = lambda x : pq[x])
            del pq[u]
            for v, w in self.adjList[u]: 
                self.relax(u, v, w, edgeTo, distTo, pq)

        return edgeTo
        
    # monotonic version 
    def monotonic_relax(self, u, v, w, edgeTo, distTo, pq):
        if u in edgeTo:
            # monotonic condition check: the weight of the shortest edge into u (edgeTo[u][2]) is < than the next weight in the path (u,v):w
            if edgeTo[u][2] < w and distTo[v] > distTo[u] + w:
                distTo[v] = distTo[u] + w
                edgeTo[v] = (u,v,w) # latest edge in shortest path to v
                pq[v] = distTo[v]

    def monotonic_dijkstra(self, source):
        edgeTo = dict()
        distTo = dict()

        for k in self.adjList.keys():
            edgeTo[k] = None
            distTo[k] = sys.maxsize
        distTo[source] = 0
        #to be able to check the monotonic condition, for each u in shortest path we maintain [from, x, weight(from,x)]; for the root node, we set from=None and w=-1
        edgeTo[source] = (None, 0, -1) 

        pq = {source : 0}
        while pq:
            u = min(pq, key = lambda x : pq[x])
            del pq[u]
            for v, w in self.adjList[u]: 
                self.monotonic_relax(u, v, w, edgeTo, distTo, pq)

        return edgeTo
    

class UnionFind:
    def __init__(self, N):
        self.parent = [i for i in range(0, N)]
        self.N = N

    def root(self, u):
        while u != self.parent[u]:
            u = self.parent[u]
        return u

    def find(self, u, v):
        return self.root(u) == self.root(v)

    def union(self, u, v):
        root_u = self.root(u)
        root_v = self.root(v)
        self.parent[root_u] = root_v #or self.parent[root_v]
        
    def findCycle(self, u, v):
        root_u = self.root(u)
        root_v = self.root(v)
        if root_u == root_v:
            return True
        else:
            self.union(root_u, root_v)

In [51]:
testedges = {(0,1):3, (0,2):2, (1,2):1, (1,4):7, (1,3):10, (3,4):9}
testG = Graph(testedges)
mst = testG.kruskalMST()
mst_2 = testG.primMST()
print(testG.mstEdges)
print(mst_2)

[(2, (2, 0)), (1, (1, 2)), (7, (4, 1)), (9, (3, 4))]
([(2, (2, 0)), (1, (1, 2)), (7, (4, 1)), (9, (3, 4))], 19)


In [53]:
edges = {(0,1):0, (0,2):50, (1,2):30, (1,4):35, (1,3):10, (3,4):15}
g = Graph(edges)
sp = g.dijkstra(0)
print("EdgeTo array using Dijkstra:", sp)
spm = g.monotonic_dijkstra(0)
print("EdgeTo array using monotonic Dijkstra:", spm)

edges = {(0,1):40, (0,2):50, (1,2):30, (1,4):20, (1,3):10, (3,4):15}
g = Graph(edges)
sp = g.dijkstra(0)
print("EdgeTo array using Dijkstra:", sp)
spm = g.monotonic_dijkstra(0)
print("EdgeTo array using monotonic Dijkstra:", spm)

EdgeTo array using Dijkstra: {0: None, 1: (0, 1, 0), 2: (1, 2, 30), 4: (3, 4, 15), 3: (1, 3, 10)}
EdgeTo array using monotonic Dijkstra: {0: (None, 0, -1), 1: (0, 1, 0), 2: (1, 2, 30), 4: (3, 4, 15), 3: (1, 3, 10)}
EdgeTo array using Dijkstra: {0: None, 1: (0, 1, 40), 2: (0, 2, 50), 4: (1, 4, 20), 3: (1, 3, 10)}
EdgeTo array using monotonic Dijkstra: {0: (None, 0, -1), 1: (0, 1, 40), 2: (0, 2, 50), 4: None, 3: None}
